In [1]:
import os

import numpy as np
import pandas as pd

import xarray as xr

In [2]:
path_d = "../temp/quantile_vars/"
path_r = "../temp/quantile_vars_model/"

vars = [ "Pressure", "Relative Humidity", "Temperature",
    "Wind Direction", "Wind Speed", "DNI", "GHI", "UVHI" ]

for v in vars:
    print(f"\n{v}")
    files = os.listdir(path_d + v + "/")
    files.sort()
    if ".DS_Store" in files: files.remove(".DS_Store")
    for f in files:
        print( f"{f[-5:-3]}", end = "," )

        with xr.open_dataset(path_d + v + "/" + f) as ds:
            df = ds.to_dataframe()
            if v in vars[-3:]:
                df = df[ df>0 ].dropna()
            latitude = df.index.get_level_values("lat").unique()
            longitude = df.index.get_level_values("lon").unique()
            time = df.index.get_level_values("time").unique()
            df = df.reset_index().set_index(["lat", "lon"]).sort_index()
            q_model = np.linspace( 1/time.shape[0], 1, time.shape[0] )

            df["q_model"] = 0.0
            for lat in latitude:
                for lon in longitude:
                    df_c = df.xs( (lat, lon) ).sort_values(v)
                    df_c["q_model"] = q_model
                    df.loc[ (lat, lon) ] = df_c

            df = df.reset_index().set_index(
                ["q_model", "lat", "lon"] ).sort_index()
            if v in vars[-3:]:
                df.to_csv(path_r + v + "/" + f[:-2] + "csv", mode = "w" )
            else:
                df.to_xarray().to_netcdf(path_r + v + "/" + f, mode = "w" )


Pressure
20,21,22,23,24,
Relative Humidity
20,21,22,23,24,
Temperature
20,21,22,23,24,
Wind Direction
20,21,22,23,24,
Wind Speed
20,21,22,23,24,
DNI
20,21,22,23,24,
GHI
20,21,22,23,24,
UVHI
20,21,22,23,24,